In [ ]:
# !pip install rerun-sdk vrs opencv-python numpy

In [1]:
!pip install -U "rerun-sdk[notebook]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 12.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: rerun-notebook
    Found existing installation: rerun-notebook 0.23.3
    Uninstalling rerun-notebook-0.23.3:
      Successfully uninstalled rerun-notebook-0.23.3


In [1]:
%gui tk

In [7]:
import time
from pathlib import Path

import cv2
import numpy as np
import rerun as rr
from pyvrs.reader import SyncVRSReader


def play_vrs_realtime(
    vrs_path: str | Path = "/home/peiyu/Projects/MR/vrs_2025_06_05/person_a.vrs",
    rgb_sid: str = "214-1",
    resize_to: tuple[int, int] | None = (960, 540),
    rotate_90: bool = True,
    play_speed: float = 1.0,          
    max_sec: float | None = None,     
):
    vrs_path = Path(vrs_path).expanduser()
    if not vrs_path.exists():
        raise FileNotFoundError(vrs_path)

    rr.init("vrs_live_stream")
    rr.spawn(title="VRS Live Viewer", memory_limit="4GB")   

    reader     = SyncVRSReader(str(vrs_path))
    cam_stream = reader.filtered_by_fields(stream_ids=rgb_sid, record_types="data")

    first_ts = None            
    wall0    = time.time()    

    try:
        for rec in cam_stream:
            if first_ts is None:
                first_ts = rec.timestamp
            rel_stream_sec = (rec.timestamp - first_ts) / play_speed
            rel_wall_sec   = time.time() - wall0
            ahead = rel_stream_sec - rel_wall_sec
            if ahead > 0:
                time.sleep(ahead)         

            if max_sec and rel_stream_sec > max_sec:
                break

            buf = rec.image_blocks[0]
            img_bgr = cv2.imdecode(np.frombuffer(buf, np.uint8), cv2.IMREAD_COLOR)
            if img_bgr is None:
                continue

            img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            if rotate_90:
                img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            if resize_to:
                img = cv2.resize(img, resize_to, interpolation=cv2.INTER_LINEAR)

            rr.set_time(
                timeline="rel_time_us",
                timestamp=int(rel_stream_sec * 1e6)
            )
            rr.log("cam/frame", rr.Image(img))

    except KeyboardInterrupt:
        print("Stop")

    print("Play End")


play_vrs_realtime()

TypeError: spawn() got an unexpected keyword argument 'title'